## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных:**
- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

### План работы<a id="plan"></a>

1. [Загрузка и обзор данных](#st1)
1. [Предобработка данных](#st2)
    - [Обработка пропусков](#st2_1)
    - [Замена типа данных](#st2_2)
    - [Обработка дубликатов](#st2_3)
    - [Лемматизация](#st2_4)
    - [Категоризация данных](#st2_5)
1. [Ответы на вопросы](#st3)
1. [Общий вывод](#st4)

### [1. Загрузка и обзор данных](#plan)<a id="st1"></a>  

In [52]:
import pandas as pd
from collections import Counter
import pymorphy2
from pymystem3 import Mystem
from tqdm import tqdm
tqdm.pandas()
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [53]:
data = pd.read_csv('01_data.csv')
print(data.info())
#display(data.head(15))

#print(data.isna().sum())

# На глаз кажется, что пропуски в 'days_employed' совпадают с пропусками в 'total_income'. Проверим:
#display(data[data['days_employed'].isna() & data['total_income'].isna()])

# Убедимся, что данные в поле 'debt' корректны:
#display(data[(data['debt'] > 1) | (data['debt'] < 0)]) # Ok

# Проверим корректность значений в поле 'children'
#display(data['children'].value_counts()) # 47 записей с отрицательным значением. И 76 записей с подозрительно большим значением (20 детей?).

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


#### Вывод

Выгрузка нуждается в предварительной обработке. 

1. В поле Количество детей `children` попадаются отрицательные значения - некорректные записи.
1. Трудовой стаж в днях `days_employed`:
   - Представлен в вещественных и порой отрицательных числах. Видимо, техническая ошибка. Если бы поле было критичным для решения поставленной задачи, нужно было бы обратиться к инженеру, готовившему выгрузку и обсудить с ним эту проблему.
   - Есть пропущенные значения (2174 - 10% данных), причём, они идут в связке с пропущенными значениями в столбце Доход в месяц `total_income` . Возможно, клиенты просто не предоставили данную информацию, т.е. пропуски неслучайны.
1. Образование `education` записано разнородно без учёта регистра. Можно вынести в словарь.
1. Семейное положение `family_status` можно вынести в словарь.
1. Поле `debt` имеет целочисленный тип, ходя по факту оно логическое.
1. Цель получения кредита `purpose`, видимо, заполнялась клиентами произвольно - данные разнородны, много синонимов. Требуется лемматизация.

### [2. Предобработка данных](#plan)<a id="st2"></a>  

#### 2.1. Обработка пропусков<a id="st2_1"></a>  

In [54]:
# Для начала удалим поле с Трудовым стажем
data = data.drop('days_employed', axis=1)

# вычислим среднее значение Дохода в месяц:
mean_total_income = data['total_income'].mean()

# Заполним пропуски в 'total_income' средним значением:
data['total_income'] = data['total_income'].fillna(mean_total_income)
# Убедимся, что пропусков больше нет: 
#print(data['total_income'].isna().value_counts())

# Удалим записи, в которых указано отрицательное количество детей:
import numpy as np
data = data.replace(to_replace = -1, value = np.nan)
data = data.dropna(subset=['children'])

print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21478 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21478 non-null  float64
 1   dob_years         21478 non-null  int64  
 2   education         21478 non-null  object 
 3   education_id      21478 non-null  int64  
 4   family_status     21478 non-null  object 
 5   family_status_id  21478 non-null  int64  
 6   gender            21478 non-null  object 
 7   income_type       21478 non-null  object 
 8   debt              21478 non-null  int64  
 9   total_income      21478 non-null  float64
 10  purpose           21478 non-null  object 
dtypes: float64(2), int64(4), object(5)
memory usage: 2.0+ MB
None


#### Вывод

Данные о Трудовом стаже выгружены с ошибкой, к тому же они не нужны для решения поставленной задачи - удалили этот столбец. Остальные необязательые поля оставим для экспериментов.
Пропуски в Информации о доходе заполнили средними значениями по столбцу.
Удалили строки с некорректными данными о Количестве детей (благо, это всего 0.2% от всей выгрузки).

#### 2.2. Замена типа данных<a id="st2_2"></a>  

In [55]:
# Приведём поле 'debt' к логическому типу данных:
try:
    data['debt'] = data['debt'].astype('bool')
except:
    print('Проверьте корректность данных в столбце "debt"')


# Вернём полю 'children' тип int:
try:
    data['children'] = data['children'].astype('int64')
except:
    print('Проверьте корректность данных в столбце "children"')
    
# Проверим:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21478 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21478 non-null  int64  
 1   dob_years         21478 non-null  int64  
 2   education         21478 non-null  object 
 3   education_id      21478 non-null  int64  
 4   family_status     21478 non-null  object 
 5   family_status_id  21478 non-null  int64  
 6   gender            21478 non-null  object 
 7   income_type       21478 non-null  object 
 8   debt              21478 non-null  bool   
 9   total_income      21478 non-null  float64
 10  purpose           21478 non-null  object 
dtypes: bool(1), float64(1), int64(4), object(5)
memory usage: 1.8+ MB
None


#### Вывод

Поле 'debt' с информацией о задолженности клиента может содержать только два значения: да или нет. Соответственно, оно должно быть логическим, а не целочисленным. Для экономии памяти и исключения некорректных значений привели его к типу boolean.
После удаления строк с отрицательным количеством детей тип поля 'children' изменился на float - исправили обратно на int.

#### 2.3. Обработка дубликатов<a id="st2_3"></a>  

In [56]:
# Приведём данные в поле 'education' к нижнему регистру и поищем дубли:
data['education'] = data['education'].str.lower()
#print(data.duplicated().sum()) # 71

# Удалим полные дубли из таблицы:
data = data.drop_duplicates().reset_index(drop=True)
print('Полных дублей в выгрузке:', data.duplicated().sum())

Полных дублей в выгрузке: 0


#### Вывод

Удалили полные дубли из выгрузки (71 запись).

#### 2.4. Лемматизация<a id="st2_4"></a>  

*Вариант с pymorphy2 (около 30 сек)*

In [57]:
morph = pymorphy2.MorphAnalyzer()

def lemmatize_purpose(row):
# Функция принимает на вход строку нашей таблицы, и возвращает лемматизированное значение столбца 'purpose' 
    text_lst = [s.strip() for s in row['purpose'].split(' ')]
    text = []

    for word in text_lst:
        p = morph.parse(word)
        text.append(p[0].normal_form)

    lemm_text = ' '.join(text)
    return lemm_text

"""
# Тестируем функцию:
test_data = data[['education', 'purpose']].head(10)
display(test_data)

test_data['purpose_lemm'] = test_data.apply(lemmatize_purpose, axis=1)
display(test_data)
"""

# Создаём новый столбец с лемматизированной Целью получения крудита ('purpose'):
data['purpose_lemm'] = data.progress_apply(lemmatize_purpose, axis=1)

100%|███████████████████████████████████████████████████████████████████████████| 21407/21407 [00:25<00:00, 824.90it/s]


*Вариант с pymystem3. Оказался намного дольше (>8 часов /Windows 10)*

In [58]:
# m = Mystem()

# def lemmatize_purpose(row):
# # Функция принимает на вход строку нашей таблицы, и возвращает лемматизированное значение столбца 'purpose' 
#     lemm_text = ' '.join(m.lemmatize(row['purpose'])) #склеиваем массив лемм, чтобы получить строку
#     return lemm_text

# """
# # Тестируем функцию:
# test_data = data[['education', 'purpose']].head(10)
# display(test_data)
# print()
# test_data['purpose_lemm'] = test_data.apply(lemmatize_purpose, axis=1)
# display(test_data)
# """

# # Создаём новый столбец с лемматизированной Целью получения крудита ('purpose'):
# data['purpose_lemm'] = data.progress_apply(lemmatize_purpose, axis=1)

In [59]:
display(data.head(10))

# Посчитаем частоту значений в общем списке лемм:
display(Counter(data['purpose_lemm']))

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemm
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875.639453,покупка жилья,покупка жильё
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080.014102,приобретение автомобиля,приобретение автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,False,145885.952297,покупка жилья,покупка жильё
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628.550329,дополнительное образование,дополнительный образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616.077870,сыграть свадьбу,сыграть свадьба
5,0,27,высшее,0,гражданский брак,1,M,компаньон,False,255763.565419,покупка жилья,покупка жильё
6,0,43,высшее,0,женат / замужем,0,F,компаньон,False,240525.971920,операции с жильем,операция с жильё
7,0,50,среднее,1,женат / замужем,0,M,сотрудник,False,135823.934197,образование,образование
8,2,35,высшее,0,гражданский брак,1,F,сотрудник,False,95856.832424,на проведение свадьбы,на проведение свадьба
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,False,144425.938277,покупка жилья для семьи,покупка жильё для семья


Counter({'покупка жильё': 643,
         'приобретение автомобиль': 459,
         'дополнительный образование': 458,
         'сыграть свадьба': 764,
         'операция с жильё': 651,
         'образование': 444,
         'на проведение свадьба': 767,
         'покупка жильё для семья': 638,
         'покупка недвижимость': 618,
         'покупка коммерческий недвижимость': 661,
         'покупка жилой недвижимость': 604,
         'строительство собственный недвижимость': 634,
         'недвижимость': 632,
         'строительство недвижимость': 619,
         'на покупка подержать автомобиль': 477,
         'на покупка свой автомобиль': 504,
         'операция с коммерческий недвижимость': 649,
         'строительство жилой недвижимость': 621,
         'жильё': 645,
         'операция с свой недвижимость': 624,
         'автомобиль': 969,
         'заняться образование': 408,
         'сделка с подержать автомобиль': 485,
         'получение образование': 441,
         'свадьба': 791,
  

#### Вывод

Оказывается, Целей кредита не так уж и много, особенно, если присмотреться повнимательнее:
- недвижимость (жильё)
- ремонт
- свадьба
- образование
- автомобиль

#### 2.5. Категоризация данных<a id="st2_5"></a>  

In [60]:
# Создадим справочник Целей кредита:
purpose = {0: 'недвижимость',
           1: 'ремонт',
           2: 'свадьба',
           3: 'образование',
           4: 'автомобиль'}
purpose_dict = pd.Series(purpose)

# Добавим в исходную выгрузку поле с индентификатором Цели кредита

def purpose_id(row):
# Функция принимает на вход строку таблицы и возвращает идентификатор Цели кредита
    if 'ремонт' in row['purpose_lemm']: return 1
    if ('недвижимость' in row['purpose_lemm']) | ('жилье' in row['purpose_lemm']) | ('жильё' in row['purpose_lemm']): return 0
    if 'свадьба' in row['purpose_lemm']: return 2
    if 'образование' in row['purpose_lemm']: return 3
    if 'автомобиль' in row['purpose_lemm']: return 4
    return print('Идентификатор Цели кредита не найден: проверьте словарь')


# Тестируем работу функции:
"""
test_data = data[['education', 'purpose_lemm']].head(10)
display(test_data)
test_data['purpose_id'] = test_data.apply(purpose_id, axis=1)
display(test_data)
"""

# Создаём новый столбец с идентификатором Цели получения крудита ('purpose_id'):
data['purpose_id'] = data.apply(purpose_id, axis=1)

#*********************** Разделим нашу выгрузку на несколько таблиц ***********************

# Основная таблица:
data_log = data[['children', 'education_id', 'family_status_id', 'debt', 'total_income', 'purpose_id']]
display(data_log.head(5))

# Словарь типов Образования:
education_dict = data[['education_id', 'education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
print('Словарь типов образования:')
display(education_dict)

# Словарь типов Семейного положения:
family_status_dict = data[['family_status_id', 'family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
print('Словарь типов Семейного положения:')
display(family_status_dict)

# Справочник Целей кредита:
print('Справочник Целей кредита:')
display(purpose_dict)

,children,education_id,family_status_id,debt,total_income,purpose_id
0,1,0,0,False,253875.639453,0
1,1,1,0,False,112080.014102,4
2,0,1,0,False,145885.952297,0
3,3,1,0,False,267628.550329,3
4,0,1,1,False,158616.077870,2


Словарь типов образования:


,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Словарь типов Семейного положения:


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Справочник Целей кредита:


0    недвижимость
1          ремонт
2         свадьба
3     образование
4      автомобиль
dtype: object

#### Вывод

Для удобства работы с выгрузкой оставим в ней только необходимые для решения задачи поля. Вынесем в отдельные словари Образование, Семейный статус и Цель получения кредита.

### [3. Ответы на вопросы](#plan)<a id="st3"></a>  

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [66]:
print('Количество детей // Всего клиентов / Клиентов с задолженностью / Доля клиентов с задолженностью, %:')
data_log_gr_by_children = data_log.groupby('children').agg(count = ('debt','count'), 
                                                           debt_count = ('debt', 'sum'))
data_log_gr_by_children['debt_part'] = (data_log_gr_by_children['debt_count'] / data_log_gr_by_children['count']) * 100
display(data_log_gr_by_children.sort_values(by='debt_part', ascending=False))

Количество детей // Всего клиентов / Клиентов с задолженностью / Доля клиентов с задолженностью, %:


,count,debt_count,debt_part
children,,,
20,76,8.0,10.526316
4,41,4.0,9.756098
2,2052,194.0,9.454191
1,4808,444.0,9.234609
3,330,27.0,8.181818
0,14091,1063.0,7.543822
5,9,0.0,0.000000


Как видно из таблицы, наличие детей само по себе увеличивает риск задолженности по кредиту. 
Выделяются клиенты с пятью детьми - среди них нет должников, но количество таких клиентов слишком мало (9 человек или 0.04% выборки), чтобы делать какие-то выводы.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [73]:
def get_family_status(fs_id):
# Функция принимает на вход идентификатор Семейного положения (int) и возвращает строку с его названием
    try:
        return family_status_dict[family_status_dict['family_status_id'] == fs_id]['family_status']
    except:
        return 'статус семейного положения не найден'

print('Семейный статус // Всего клиентов / Клиентов с задолженностью / Доля клиентов с задолженностью, %:')
data_log_gr_by_fam = data_log.groupby('family_status_id', as_index = False).agg({'debt': ['count', 'sum']})
data_log_gr_by_fam['debt_part'] = (data_log_gr_by_fam['debt']['sum'] / data_log_gr_by_fam['debt']['count']) * 100

display(data_log_gr_by_fam.sort_values(by='debt_part', ascending = False))
display(family_status_dict)

Семейный статус // Всего клиентов / Клиентов с задолженностью / Доля клиентов с задолженностью, %:


family_status_id   debt        debt_part
                    count    sum          
4                4   2805  274.0  9.768271
1                1   4146  388.0  9.358418
0                0  12310  930.0  7.554833
3                3   1191   85.0  7.136860
2                2    955   63.0  6.596859

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


#### Вывод

Выстроилась последовательность, похожая на жизненный путь человека. Чем ближе клиент к официальному браку и далее, тем меньше вероятность наличия у него задолженности по кредиту. Возможно, эта зависимость связана не с семейным положением, как таковым, а с возрастом человека.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [71]:
"""
Функция принимает на вход размер дохода и возвращает его Уровень по следующей логике:
    доход не более 50 000 - низкий уровень дохода
    доход 50 000 - 250 000 - средний уровень дохода
    доход более 250 000 - высокий уровень дохода
"""
def income_type(income):
    if income <= 50000: return 'низкий'
    if 50000 < income <= 250000: return 'средний'
    if income > 250000: return 'высокий'    
    
data_log['income_type'] = data_log['total_income'].apply(income_type)
#print(data_log['income_type'].value_counts())

print('Уровень дохода // Всего клиентов / Клиентов с задолженностью / Доля клиентов с задолженностью, %:')
data_log_gr_by_income = data_log.groupby('income_type').agg(count = ('debt', 'count'), 
                                                            debt_count = ('debt', 'sum'))
data_log_gr_by_income['debt_part'] = (data_log_gr_by_income['debt_count'] / data_log_gr_by_income['count']) * 100

display(data_log_gr_by_income.sort_values(by='debt_part', ascending = False))

Уровень дохода // Всего клиентов / Клиентов с задолженностью / Доля клиентов с задолженностью, %:


,count,debt_count,debt_part
income_type,,,
средний,18229,1523.0,8.354819
высокий,2807,194.0,6.911293
низкий,371,23.0,6.199461


#### Вывод

Всё, конечно, зависит от того, где находятся границы между уровнями дохода. Если принять, что доход не более 50000 считается низким, между 50000 и 250000 - средним, и более 250000 - высоким, то получаем, что самый высокий процент задолженности по кредиту надлюдается среди клиентов со средним уровнем дохода. Как видно из таблицы, люди с таким уровнем дохода чаще других обращаются в банк за кредитом.

- Как разные цели кредита влияют на его возврат в срок?

In [82]:
def get_purpose(purpose_id):
# Функция принимает на вход идентификатор Цели кредита (int) и возвращает строку с её названием
    try:
        return purpose_dict[purpose_id]
    except:
        return 'наименование Цели кредита не найдено'

print('Цель кредита // Всего клиентов / Клиентов с задолженностью / Доля клиентов с задолженностью, %:')
data_log_gr_by_purp = data_log.groupby('purpose_id').agg(count = ('debt', 'count'), 
                                                         debt_count = ('debt', 'sum'))
data_log_gr_by_purp['debt_part'] = (data_log_gr_by_purp['debt_count'] / data_log_gr_by_purp['count']) * 100

#data_log_gr_by_fam['family_status'] = data_log_gr_by_fam['family_status_id'].apply(get_family_status)

display(data_log_gr_by_purp.sort_values(by='debt_part', ascending = False))
display(purpose_dict)

Цель кредита // Всего клиентов / Клиентов с задолженностью / Доля клиентов с задолженностью, %:


,count,debt_count,debt_part
purpose_id,,,
4,4295,402.0,9.359721
3,4003,370.0,9.243068
2,2322,186.0,8.010336
0,10181,747.0,7.337197
1,606,35.0,5.775578


0    недвижимость
1          ремонт
2         свадьба
3     образование
4      автомобиль
dtype: object

#### Вывод

Больше всего должников среди людей, взявших кредит на автомобиль и образование, чуть меньше доля долгов среди сыгравших свадьбу и взявших кредит на недвижимость. Самый "безопасный" для банка продукт - кредит на ремонт. 
Получившаяся последовательность снова напоминает жизненный путь человека :)

### [4. Общий вывод](#plan)<a id="st4"></a>  

In [80]:
# Проверим гипотезу, что вероятность наличия задолженности по кредиту падает с увеличением возраста клиента:
def age_group(years):
    if  0 < years <= 18: return 'дети'
    if 18 < years <= 30: return 'молодые'
    if 30 < years <= 45: return 'возмужалые'
    if 45 < years <= 60: return 'пожилые'
    if 61 < years <= 75: return 'старые'
    if years > 75: return 'долговечные'
    return 'не определено'

data['age_group'] = data['dob_years'].apply(age_group)
print('Возраст // Всего клиентов / Клиентов с задолженностью / Доля клиентов с задолженностью, %:')
data_gr_by_age = data.groupby('age_group').agg(count = ('debt','count'), 
                                               debt_count = ('debt', 'sum'))
data_gr_by_age['debt_part'] = (data_gr_by_age['debt_count'] / data_gr_by_age['count']) * 100
display(data_gr_by_age.sort_values(by='debt_part', ascending = False))

#data.pivot_table(index='family_status',columns='children',values='debt',aggfunc=['mean'])

Возраст // Всего клиентов / Клиентов с задолженностью / Доля клиентов с задолженностью, %:


,count,debt_count,debt_part
age_group,,,
молодые,3711,403.0,10.859607
возмужалые,8463,761.0,8.992083
пожилые,7010,467.0,6.661912
старые,1769,91.0,5.144149
не определено,454,18.0,3.964758


Итак, отвечая на вопрос "влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок", можно сказать следующее:

1) Вероятность наличия задолженности падает вместе с традиционным изменением семейного положения человека. Холостые/незамужние клиенты имеют больше долгов, чем живущие в гражданском браке. Женатые/замужние ответственнее, но всё же менее надёжны, чем люди в разводе. А меньше всего задолженностей у вдовцов/вдов.

2) Наличие детей само по себе влияет на вероятность наличия задолженности по кредиту - она становится выше. При этом количество детей у клиента на факт погашения кредита в срок не влияет.

Очевидно, что есть зависимость между возрастом клиента и наличием у него задолженности: чем старше клиент, тем эта вероятность ниже. Возможно, семейное положение и возраст влияют на факт погашения кредита в срок одинаково, т.к. являются сторонами одной медали.